In [21]:
from dlinputs import tarrecords
reload(tarrecords)

<module 'dlinputs.tarrecords' from 'dlinputs/tarrecords.py'>

In [22]:
stream = open("testdata/imagenet-000000.tgz")
data = tarrecords.tariterator(stream)
samples = list(data)
assert len(samples)==47
assert samples[0]["__key__"] == "./10"
assert set(samples[3].keys()) == set("__key__ cls png xml wnid".split())
assert samples[-1]["png"].shape == (400, 300, 3)

In [24]:
stream = open("/tmp/test.tgz", "wb")
sink = tarrecords.TarWriter(stream)
for sample in samples:
    sink.write(sample)
sink.close()
stream.close()
!ls -l /tmp/test.tgz

-rw-rw-r-- 1 tmb tmb 12933749 Mar 15 14:08 /tmp/test.tgz


In [25]:
stream = open("/tmp/test.tgz")
data = tarrecords.tariterator(stream)
samples = list(data)
assert len(samples)==47
assert samples[0]["__key__"] == "./10"
assert set(samples[3].keys()) == set("__key__ cls png xml wnid".split())
assert samples[-1]["png"].shape == (400, 300, 3)